In [188]:
import numpy as np
import pandas as pd
import os
import json
from collections import Counter
from ast import literal_eval

In [189]:
business_df = pd.read_json('data/yelp_reviews/yelp_academic_dataset_business.json', lines=True)

In [190]:
food_data = business_df[business_df.categories.str.lower().str.contains('food|restaurants') == True]
food_data = food_data[food_data.is_open == 1]
food_data

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,1,"{'BusinessParking': '{'garage': False, 'street...","Specialty Food, Food, Coffee & Tea, Coffee Roa...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-14:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [191]:


counter = Counter() 

def count(d, counter):
    if d is not None:
        counter.update(list(d.keys()))
        
food_data['attributes'].apply(lambda d : count(d, counter))

counter.most_common()


[('RestaurantsTakeOut', 40458),
 ('BusinessParking', 38439),
 ('BusinessAcceptsCreditCards', 37881),
 ('RestaurantsDelivery', 37838),
 ('RestaurantsPriceRange2', 36438),
 ('BikeParking', 32271),
 ('OutdoorSeating', 30141),
 ('WiFi', 28610),
 ('RestaurantsReservations', 28443),
 ('HasTV', 28296),
 ('Caters', 28260),
 ('Ambience', 27854),
 ('GoodForKids', 26892),
 ('RestaurantsGoodForGroups', 26674),
 ('Alcohol', 26392),
 ('RestaurantsAttire', 24478),
 ('NoiseLevel', 23098),
 ('GoodForMeal', 21747),
 ('RestaurantsTableService', 14859),
 ('WheelchairAccessible', 12957),
 ('DogsAllowed', 10895),
 ('HappyHour', 10133),
 ('DriveThru', 6883),
 ('BusinessAcceptsBitcoin', 5844),
 ('Music', 4463),
 ('ByAppointmentOnly', 4208),
 ('BYOB', 3935),
 ('BestNights', 3686),
 ('CoatCheck', 3594),
 ('Corkage', 2986),
 ('Smoking', 2736),
 ('GoodForDancing', 2674),
 ('BYOBCorkage', 818),
 ('AgesAllowed', 83),
 ('AcceptsInsurance', 64),
 ('Open24Hours', 22),
 ('DietaryRestrictions', 20),
 ('RestaurantsCounte

In [192]:

attribute_counter = Counter() 

def count_options_explicit(d, counter):
    if d is not None:

        entries = list(d.keys())

        temp = []
        for entry in entries:
            if "{" not in d[entry]:
                if ('u\'' in d[entry]): d[entry] = d[entry][1:]
                temp.append(f'{entry}: {d[entry]}')
            else :
                sub_d = literal_eval(d[entry])
                temp += [f'{str(entry)}: {str(sub_key)} : {str(sub_d[sub_key])}' for sub_key in list(sub_d.keys())]
                
        counter.update(temp)
        
food_data['attributes'].apply(lambda d : count_options_explicit(d, attribute_counter))

None #Prevents cell from showing output


In [193]:
#Number of relevant features
len(attribute_counter)

217

In [194]:
#Filters for top 75% of features and stores them as a list
attributes = counter.most_common()[:int(len(counter) * .5)]
attributes = [attribute[0] for _, attribute in enumerate(attributes)]
attributes

['RestaurantsTakeOut',
 'BusinessParking',
 'BusinessAcceptsCreditCards',
 'RestaurantsDelivery',
 'RestaurantsPriceRange2',
 'BikeParking',
 'OutdoorSeating',
 'WiFi',
 'RestaurantsReservations',
 'HasTV',
 'Caters',
 'Ambience',
 'GoodForKids',
 'RestaurantsGoodForGroups',
 'Alcohol',
 'RestaurantsAttire',
 'NoiseLevel',
 'GoodForMeal',
 'RestaurantsTableService']

In [195]:
for i, attribute in enumerate(attributes):
    print(f'{i}: {attribute}')

0: RestaurantsTakeOut
1: BusinessParking
2: BusinessAcceptsCreditCards
3: RestaurantsDelivery
4: RestaurantsPriceRange2
5: BikeParking
6: OutdoorSeating
7: WiFi
8: RestaurantsReservations
9: HasTV
10: Caters
11: Ambience
12: GoodForKids
13: RestaurantsGoodForGroups
14: Alcohol
15: RestaurantsAttire
16: NoiseLevel
17: GoodForMeal
18: RestaurantsTableService


In [250]:
attributes = counter.most_common()[:int(len(counter) * .5)]
attributes = [attribute[0] for _, attribute in enumerate(attributes)]

unkown = "NaN"
def process_dict(idx, scaffold, options):
    dictionary = literal_eval(scaffold[idx]) if type(scaffold[idx]) == str and "{" in scaffold[idx] else {}
    return [dictionary[option] if option in dictionary else unkown for option in options]

parking_options = ['garage', 'street', 'validated', 'lot', 'valet']
parking_idx = 1
ambience_options = ['romantic', 'intimate', 'touristy', 'hipster', 'divey', 'classy', 'trendy', 'upscale', 'casual']
ambience_idx = 11
meal_options = ['dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast']
meal_idx = 17

attribute_data = []


none_count = 0
for id, stars, lat, long, cnt, postal, row in zip(food_data.business_id, food_data.stars, food_data.latitude, food_data.longitude,food_data.review_count, food_data.postal_code, food_data.attributes):
    if row is None:
        none_count += 1
        temp = ["unknown" for _,__ in enumerate(attributes)]
        temp = [id] + temp
    else:

        temp = [row[att] if att in row else unkown for att in attributes]

        builder = temp[:parking_idx] + process_dict(parking_idx, temp, parking_options)
        builder += temp[parking_idx + 1 : ambience_idx]
        builder += process_dict(ambience_idx, temp, ambience_options)
        builder += temp[ambience_idx + 1: meal_idx]
        builder += process_dict(meal_idx, temp, meal_options)
        builder += temp[meal_idx + 1:]
        builder = [id, stars, cnt, lat, long, postal] + builder


    attribute_data.append(tuple([build.replace("'", "") if type(build) == str else build for build in builder]))

attributes_builder = ["buisness_id", "Stars", 'review_count', 'latitude', 'longitude', 'postal_code'] 
attributes_builder += attributes[:parking_idx] + parking_options + attributes[parking_idx + 1:ambience_idx] 
attributes_builder += ambience_options + attributes[ambience_idx + 1: meal_idx]
attributes_builder += meal_options + attributes[meal_idx + 1:]
attributes = [elem.lower() for elem in attributes_builder]

attributes_df = pd.DataFrame(attribute_data, columns=attributes)
attributes_df

,buisness_id,stars,review_count,latitude,longitude,postal_code,restaurantstakeout,garage,street,validated,...,alcohol,restaurantsattire,noiselevel,dessert,latenight,lunch,dinner,brunch,breakfast,restaurantstableservice
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,39.955505,-75.155564,19107,True,False,True,False,...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,40.338183,-75.471659,18054,True,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CF33F8-E6oudUQ46HnavjQ,2.0,6,36.269593,-87.058943,37015,True,NaN,NaN,NaN,...,none,casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,10,36.208102,-86.768170,37207,True,False,False,False,...,none,casual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,eEOYSgkmpB90uNA7lDOMRA,4.0,10,27.955269,-82.456320,33602,NaN,False,False,False,...,none,NaN,NaN,False,False,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44577,l9eLGG9ZKpLJzboZq-9LRQ,3.0,11,39.925656,-75.310344,19018,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44578,cM6V90ExQD6KMSU3rRB5ZA,4.0,33,43.615401,-116.284689,83704,False,False,False,False,...,none,casual,loud,False,False,False,False,False,False,NaN
44579,1jx1sfgjgVg0nM6n3p0xWA,4.5,41,32.409552,-110.943073,85737,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44580,WnT9NIzQgLlILjPT0kEcsQ,4.5,35,39.935982,-75.158665,19147,True,None,True,None,...,none,casual,NaN,True,None,True,True,None,None,True


In [251]:
for col in attributes_df.columns.tolist()[6:]:
    print(col)
    print(attributes_df[col].value_counts())

restaurantstakeout
True     36682
NaN       3472
False     3264
None      1164
Name: restaurantstakeout, dtype: int64
garage
False    34869
NaN       6275
True      1743
Name: garage, dtype: int64
street
False    25805
True      9473
NaN       6278
Name: street, dtype: int64
validated
False    35962
NaN       6278
True       386
Name: validated, dtype: int64
lot
True     21203
False    15030
NaN       6278
Name: lot, dtype: int64
valet
False    37518
NaN       6277
True       787
Name: valet, dtype: int64
businessacceptscreditcards
True     37294
NaN       6097
False     1171
None        20
Name: businessacceptscreditcards, dtype: int64
restaurantsdelivery
True     26393
False     9862
NaN       6142
None      2185
Name: restaurantsdelivery, dtype: int64
restaurantspricerange2
2       18738
1       16825
NaN      7561
3        1274
4         175
None        9
Name: restaurantspricerange2, dtype: int64
bikeparking
True     24255
NaN      11783
False     8508
None        36
Name: bikepar

In [252]:
print(attributes_df.columns.tolist())

['buisness_id', 'stars', 'review_count', 'latitude', 'longitude', 'postal_code', 'restaurantstakeout', 'garage', 'street', 'validated', 'lot', 'valet', 'businessacceptscreditcards', 'restaurantsdelivery', 'restaurantspricerange2', 'bikeparking', 'outdoorseating', 'wifi', 'restaurantsreservations', 'hastv', 'caters', 'romantic', 'intimate', 'touristy', 'hipster', 'divey', 'classy', 'trendy', 'upscale', 'casual', 'goodforkids', 'restaurantsgoodforgroups', 'alcohol', 'restaurantsattire', 'noiselevel', 'dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast', 'restaurantstableservice']


In [272]:
default_attributes = {
    'RestaurantsDelivery':False,
    'garage':False,
    'street':False,
    'validated':False,
    'lot':False,
    'valet':False,
    'BusinessAcceptsCreditCards': True,
    'RestaurantsPriceRange2': 2,
    'RestaurantsDelivery': True,
    'OutdoorSeating': False,
    'BikeParking': False,
    'RestaurantsReservations': True,
    'HasTV': False,
    'GoodForKids': True,
    'romantic': False,
    'intimate': False,
    'touristy': False,
    'hipster': False,
    'divey': False,
    'classy': False,
    'trendy': False,
    'upscale': False,
    'casual': True,
    'WiFi': 'no',
    'RestaurantsGoodForGroups': True,
    'Alcohol': 'none',
    'Caters': False,
    'RestaurantsTakeout': False,
    'RestaurantsAttire': 'none',
    'NoiseLevel': 'average',
    'dessert': True,
    'latenight': False,
    'lunch': True,
    'dinner': True,
    'brunch': False,
    'breakfast': False,
    'RestaurantsTableService': True
}

def str_to_bool(s):
    if (type(s) is str):
        return s == "True"
    else: return s

for attribute, value in default_attributes.items():
    attributes_df[attribute.lower()] = attributes_df[attribute.lower()].replace([unkown], value)
    attributes_df[attribute.lower()] = attributes_df[attribute.lower()] =  attributes_df[attribute.lower()] = attributes_df[attribute.lower()].map(lambda x : value if pd.isna(x) else x)
    if type(value) is bool: attributes_df[attribute.lower()] = attributes_df[attribute.lower()].replace(['None'], False)
    attributes_df[attribute.lower()] = attributes_df[attribute.lower()].map(lambda s : str_to_bool(s) if type(value) is bool else s)

attributes_df['wifi'] = attributes_df['wifi'].replace(['None'], 'no')
attributes_df['alcohol'] = attributes_df['wifi'].replace(['None'], 'none')
attributes_df['RestaurantsAttire'] = attributes_df['restaurantsattire'].replace(['None'], 'none')




In [273]:
type(True)

bool

In [274]:
attributes_df

,buisness_id,stars,review_count,latitude,longitude,postal_code,restaurantstakeout,garage,street,validated,...,restaurantsattire,noiselevel,dessert,latenight,lunch,dinner,brunch,breakfast,restaurantstableservice,RestaurantsAttire
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,39.955505,-75.155564,19107,True,False,True,False,...,none,average,True,False,True,True,False,False,True,none
1,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,40.338183,-75.471659,18054,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
2,CF33F8-E6oudUQ46HnavjQ,2.0,6,36.269593,-87.058943,37015,True,False,False,False,...,casual,average,True,False,True,True,False,False,False,casual
3,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,10,36.208102,-86.768170,37207,True,False,False,False,...,casual,average,True,False,True,True,False,False,False,casual
4,eEOYSgkmpB90uNA7lDOMRA,4.0,10,27.955269,-82.456320,33602,False,False,False,False,...,none,average,False,False,False,False,False,False,True,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44577,l9eLGG9ZKpLJzboZq-9LRQ,3.0,11,39.925656,-75.310344,19018,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
44578,cM6V90ExQD6KMSU3rRB5ZA,4.0,33,43.615401,-116.284689,83704,False,False,False,False,...,casual,loud,False,False,False,False,False,False,True,casual
44579,1jx1sfgjgVg0nM6n3p0xWA,4.5,41,32.409552,-110.943073,85737,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
44580,WnT9NIzQgLlILjPT0kEcsQ,4.5,35,39.935982,-75.158665,19147,True,False,True,False,...,casual,average,True,False,True,True,False,False,True,casual


In [275]:
for col in attributes_df.columns.tolist()[6:]:
    print(col)
    print(attributes_df[col].value_counts())

restaurantstakeout
True     36682
False     7900
Name: restaurantstakeout, dtype: int64
garage
False    42839
True      1743
Name: garage, dtype: int64
street
False    35109
True      9473
Name: street, dtype: int64
validated
False    44196
True       386
Name: validated, dtype: int64
lot
False    23379
True     21203
Name: lot, dtype: int64
valet
False    43795
True       787
Name: valet, dtype: int64
businessacceptscreditcards
True     43391
False     1191
Name: businessacceptscreditcards, dtype: int64
restaurantsdelivery
True     32535
False    12047
Name: restaurantsdelivery, dtype: int64
restaurantspricerange2
2       18738
1       16825
2        7561
3        1274
4         175
None        9
Name: restaurantspricerange2, dtype: int64
bikeparking
True     24255
False    20327
Name: bikeparking, dtype: int64
outdoorseating
False    30568
True     14014
Name: outdoorseating, dtype: int64
wifi
no      27849
free    16555
paid      178
Name: wifi, dtype: int64
restaurantsreservations


In [276]:
type(business_df)


pandas.core.frame.DataFrame

In [277]:
food_data.to_json("data/cleaned/yelp_academic_dataset_restaurant.json")

In [278]:
check_df = pd.read_json('data/cleaned/yelp_academic_dataset_restaurant.json')
check_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150328,1jx1sfgjgVg0nM6n3p0xWA,Savaya Coffee Market,11177 N Oracle Rd,Oro Valley,AZ,85737,32.409552,-110.943073,4.5,41,1,"{'BusinessParking': '{'garage': False, 'street...","Specialty Food, Food, Coffee & Tea, Coffee Roa...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-14:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [279]:
attributes_df.to_json("data/cleaned/yelp_academic_dataset_restaurant_att.json")

In [280]:
test = pd.read_json('data/cleaned/yelp_academic_dataset_restaurant_att.json') 
test

,buisness_id,stars,review_count,latitude,longitude,postal_code,restaurantstakeout,garage,street,validated,...,restaurantsattire,noiselevel,dessert,latenight,lunch,dinner,brunch,breakfast,restaurantstableservice,RestaurantsAttire
0,MTSW4McQd7CbVtyjqoe9mw,4.0,80,39.955505,-75.155564,19107,True,False,True,False,...,none,average,True,False,True,True,False,False,True,none
1,mWMc6_wTdE0EUBKIGXDVfA,4.5,13,40.338183,-75.471659,18054,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
2,CF33F8-E6oudUQ46HnavjQ,2.0,6,36.269593,-87.058943,37015,True,False,False,False,...,casual,average,True,False,True,True,False,False,False,casual
3,bBDDEgkFA1Otx9Lfe7BZUQ,1.5,10,36.208102,-86.768170,37207,True,False,False,False,...,casual,average,True,False,True,True,False,False,False,casual
4,eEOYSgkmpB90uNA7lDOMRA,4.0,10,27.955269,-82.456320,33602,False,False,False,False,...,none,average,False,False,False,False,False,False,True,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44577,l9eLGG9ZKpLJzboZq-9LRQ,3.0,11,39.925656,-75.310344,19018,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
44578,cM6V90ExQD6KMSU3rRB5ZA,4.0,33,43.615401,-116.284689,83704,False,False,False,False,...,casual,loud,False,False,False,False,False,False,True,casual
44579,1jx1sfgjgVg0nM6n3p0xWA,4.5,41,32.409552,-110.943073,85737,True,False,False,False,...,none,average,True,False,True,True,False,False,True,none
44580,WnT9NIzQgLlILjPT0kEcsQ,4.5,35,39.935982,-75.158665,19147,True,False,True,False,...,casual,average,True,False,True,True,False,False,True,casual
